In [148]:
import pymongo
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["story_database"]  # Database name
collection = db["short_stories"]

documents = collection.find()

df_raw = pd.DataFrame(list(documents))
df_raw = df_raw[["text", "label"]]
df_raw.head(5)

,text,label
0,Transcriber's Note:\nEvery effort has been mad...,0
1,"“In a few moments Marianne, Solomin, Paul,\n\n...",0
2,“Marianne knelt beside the sofa.… Nezhdanof\n\...,0
3,"“Solomin raised Marianne's hand, her head\n\nl...",0
4,"“He now was no longer, but the hands of\n\nSol...",0


In [149]:
# Define the cleaning function
def clean_text(text):

    text = text.replace('\n', ' ') 
    text = re.sub(r'=', '', text)  

    # Keep only English alphabet characters and spaces 
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

df_raw['cleaned_text'] = df_raw['text'].apply(clean_text)

df = df_raw[["cleaned_text", "label"]]

# Vectorize the data


In [150]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

df_shuffled = df.sample(frac=1).reset_index(drop=True)


X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

cleaned_data = (classification_report(y_test, y_pred))
print(cleaned_data)

              precision    recall  f1-score   support

           0       1.00      0.63      0.78        41
           1       0.75      1.00      0.86        46

    accuracy                           0.83        87
   macro avg       0.88      0.82      0.82        87
weighted avg       0.87      0.83      0.82        87



Wow this was a bit anticlimactic, lets mess the data up see if we get the same results

# Lets not clean the data!

In [151]:
df_shuffled = df_raw.sample(frac=1).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.68      0.81        37
           1       0.81      1.00      0.89        50

    accuracy                           0.86        87
   macro avg       0.90      0.84      0.85        87
weighted avg       0.89      0.86      0.86        87



# Lets add data, fakenews!

In [152]:
fakenews = pd.read_csv("data/fake_news.csv")
fakenews.columns = ["cleaned_text", "label"]

In [153]:
data = pd.concat([fakenews, df], ignore_index=True)


In [154]:
df_shuffled = data.sample(frac=1).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.88      0.94        34
           1       0.82      0.96      0.88        52
           2       0.93      0.84      0.89        51

    accuracy                           0.90       137
   macro avg       0.92      0.90      0.90       137
weighted avg       0.91      0.90      0.90       137



In [155]:
confusion_matrix(y_test, y_pred)

array([[30,  3,  1],
       [ 0, 50,  2],
       [ 0,  8, 43]], dtype=int64)

## Is the vectorizer that good?

In [156]:
bbc_news = pd.read_csv("data/bbc_news.csv")
bbc = bbc_news[["cleaned_text", "category_encoded"]]
bbc.columns =  ["cleaned_text", "label"]

In [157]:
data = pd.concat([bbc, data], ignore_index=True)

In [158]:
data["label"].value_counts()

label
3    100
6    100
5    100
7    100
4    100
2    100
1    100
0     74
Name: count, dtype: int64

In [160]:
df_shuffled = data.sample(frac=1).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.89      0.94        18
           1       0.92      0.92      0.92        38
           2       0.95      0.72      0.82        29
           3       0.93      0.97      0.95        29
           4       0.93      0.90      0.92        31
           5       0.79      1.00      0.88        26
           6       0.89      1.00      0.94        31
           7       1.00      0.94      0.97        31

    accuracy                           0.92       233
   macro avg       0.93      0.92      0.92       233
weighted avg       0.93      0.92      0.92       233



In [161]:
confusion_matrix(y_test, y_pred)

array([[16,  2,  0,  0,  0,  0,  0,  0],
       [ 0, 35,  0,  0,  0,  0,  3,  0],
       [ 0,  1, 21,  0,  1,  6,  0,  0],
       [ 0,  0,  0, 28,  1,  0,  0,  0],
       [ 0,  0,  0,  1, 28,  1,  1,  0],
       [ 0,  0,  0,  0,  0, 26,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 31,  0],
       [ 0,  0,  1,  1,  0,  0,  0, 29]], dtype=int64)

## Increasing the datas dimensions by adding more data as well as adding more things to classify is instead of worsening the model its getting improved?
The TfidfVectorizer proves to be an excellent tool for vectorizing text data, as it does a near-perfect job of capturing the essence of the information. However, it's important to note that a vectorizer like TfidfVectorizer thrives on large datasets. While adding more labels was part of the improvement, it was the accompanying increase in data that enabled the creation of a more complex sparse matrix, allowing the model to perform more efficiently and effectively.